In [ ]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL']='2'
os.environ["RAY_IGNORE_UNHANDLED_ERRORS"] = "1"
os.environ["TUNE_DISABLE_STRICT_METRIC_CHECKING"] = "1"
os.environ["RAY_memory_monitor_refresh_ms"] = "0" # do not kill raylet if low on memmory
os.environ["RAY_TASK_MAX_RETRIES"] = "2"

os.environ["TUNE_PLACEMENT_GROUP_AUTO_DISABLED"] = "1" #neded only when running local

In [ ]:
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import TimeDistributed, LSTM, Dense, Dropout, Flatten, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau, ModelCheckpoint, EarlyStopping
from tensorflow.keras.regularizers import l1, l2, l1_l2

In [ ]:
import ray
from ray import train, tune, air
from ray.tune.schedulers import AsyncHyperBandScheduler

In [ ]:
import matplotlib.pyplot as plt
#%matplotlib notebook
%matplotlib inline

In [ ]:
import mlflow
from mlflow.models import infer_signature
from ray.air.integrations.mlflow import setup_mlflow

In [ ]:
import uuid
import time
import datetime

In [ ]:
modelTypeName = "LSTM_AE_valueMinMaxScaled"

In [ ]:
dataColumnName = 'valueMinMaxScaled'

In [ ]:
MLFLOW_URI='http://localhost:8080/'

In [ ]:
param_space={
    "n_epochs": tune.choice([200]),
    "n_layers": tune.choice([1, 3, 5, 10]),
    "n_dense_layers": tune.choice([1, 3, 5]), 
    "learning_rate": tune.choice([0.003, 0.001]),
    "activation": tune.choice(['tanh', 'relu']), 
    "n_neurons": tune.choice([32, 64, 96, 128, 256]),
    "T": tune.choice([5, 10, 15, 20]),
    "dense_dp": tune.choice([0.2, 0.3, 0.4]),
}

In [ ]:
%run '../lib/utils_anomaly_detection.ipynb'

In [ ]:
import pandas as pd

In [ ]:
# load datasets
data = pd.read_csv('../../data/rq_1-3_train_test/2024-01-15_10-51-45__2024-01-15_14-26-45_load-gen-msg-w-spikes-10s-rate.csv', 
                   index_col=['EventDateTime'], parse_dates=['EventDateTime'])
dataLatency = pd.read_csv('../../data/rq_1-3_train_test/2024-01-15_10-51-45__2024-01-15_14-26-45_load-gen-avg-latency-10s-rate.csv', 
                          index_col='EventDateTime', parse_dates=['EventDateTime'])

In [ ]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler

In [ ]:
%run '../lib/prepareDataSet.ipynb'

In [ ]:
from sklearn.metrics import r2_score, mean_absolute_percentage_error, mean_squared_error, mean_absolute_error
from keras.models import load_model, save_model

In [ ]:
start_time = datetime.datetime.now()

In [ ]:
"""
 This function uses one Tensor formatted input from the X_test dataset in order to predict ahead by a number
 of given steps. Then it readjusts by using another true value from X_test (as the nex index) before starting
 a new prediction cycle
"""
def generate_nsteps_forecast(x_test, y_test, nn_model, enc_model, pred_ahead):
    max_len = x_test.shape[0]
    y_pred = []
    last_x = x_test[0]
    dim_shape = last_x.shape[1]
    index = 0
    ae_errors_at_p = []
    se_errors_at_p = []    
    while len(y_pred) < max_len:
        sequence = 0
        while sequence < pred_ahead:
            try:
                x_crt_input = last_x.reshape(1, -1, dim_shape)
                p_vector = nn_model.predict(x_crt_input, verbose=0)
                p = p_vector[0,0] # 1x1 array -> scalar
            except:
                print(f'Prediction error for x={x_crt_input} at sequence={sequence} for start index={index} when pred_ahead={pred_ahead}')
                print(f'Model config was:{nn_model.get_config()}')
                p = 0
                
            # update the predictions list
            y_pred.append(p)

            #prepare new_x cointaing prediction p for encoding
            new_x = last_x[:,0] # the first column contains the original input x from X_train
            new_x = np.roll(new_x, -1)
            new_x[-1] = p
            
            try:
                x_vector_ = np.concatenate([new_x.reshape(1, -1, D), 
                                 enc_model.predict(new_x.reshape(1, -1, D), verbose=0)], 
                                axis = -1)
                x_ = x_vector_[0][0] # x_ is 3D we need only the last dimension as a row
            except:
                x_ = 0

            # make the new input
            last_x = np.roll(last_x, -1)
            last_x[-1] = x_
            
            # increase index for the next run
            sequence += 1

        index += sequence
        if index < max_len:
            last_x = x_test[index]
            ae_errors_at_p.append(np.absolute(p - y_test[index-1]))
            se_errors_at_p.append(np.square(p - y_test[index-1]))
            #print(f"Arrived at index = {index} of {max_len} with value X={last_x}")
    
    if len(y_pred) > max_len:
        # predicted too much, cutoff the tail
        y_pred = y_pred[0:max_len]
        
    y_pred_array = np.array(y_pred)
    avg_ae = np.mean(ae_errors_at_p)
    avg_se = np.mean(se_errors_at_p)
    return (y_pred_array, avg_ae, avg_se)

In [ ]:
def create_autoencoder(config, X_train, Y_train, X_test, Y_test, autoencoder_name):
    inputs_ae = Input(shape=(config['T'],D))
    encoded_ae = LSTM(config['n_neurons'], return_sequences=True, activation='relu')(inputs_ae, training=True)
    decoded_ae = LSTM(config['n_neurons'], return_sequences=True, activation='relu')(encoded_ae, training=True)
    out_ae = TimeDistributed(Dense(1))(decoded_ae)

    sequence_autoencoder = Model(inputs_ae, out_ae)
    sequence_autoencoder.compile(loss='mse', optimizer=Adam(learning_rate=config['learning_rate']))

    es_ae = EarlyStopping(monitor='val_loss', mode='min', verbose=0, patience=5, restore_best_weights=True)
    mc_ae = ModelCheckpoint(autoencoder_name, monitor='val_loss', mode='auto', save_best_only=True)
    
    results = sequence_autoencoder.fit(X_train, Y_train, epochs=config['n_epochs'], validation_data=(X_test, Y_test), 
                                       callbacks=[es_ae, mc_ae], verbose = 0)
    
    encoder = Model(inputs_ae, encoded_ae)
    
    return encoder, results

In [ ]:
def create_forecaster(config, x_train_shape):
    
    forecaster = Sequential()
    forecaster.add(Input(shape=x_train_shape))
    for i in range(config["n_layers"] - 1):
        forecaster.add(LSTM(config["n_neurons"], activation=config['activation'],return_sequences=True))
        
    forecaster.add(LSTM(config["n_neurons"], activation=config['activation'],return_sequences=False))
    
    for i in range(config["n_dense_layers"]-1):
        forecaster.add(Dense(config["n_neurons"], activation="relu"))
        forecaster.add(Dropout(rate=config["dense_dp"]))

    forecaster.add(Dense(1))
            
    forecaster.compile(loss='mse', optimizer=Adam(learning_rate=config['learning_rate']))
    
    return forecaster

In [ ]:
def train_model(config):
    
    n_neurons = config['n_neurons']
    n_epochs = config['n_epochs']
    learning_rate = config['learning_rate']
    T = config['T']

    model_exp =  f'{modelTypeName}'
    UUID = uuid.uuid4().hex

    mlflow_exp_name = f'{modelTypeName}-{UUID}-T_{T}-LY_{config["n_layers"]}-DLY_{config["n_dense_layers"]}-NN_{n_neurons}'

    import mlflow

    mlflow.set_tracking_uri(MLFLOW_URI)
    mlflow.set_registry_uri(MLFLOW_URI)
    
    X_train, Y_train, X_test, Y_test = prepare_dataset(dataFrame[dataColumnName], T)
    
    model_name = modelTypeName + "-T_" + str(T) + "-LY_" + str(config['n_layers']) + "-DLY_" + str(config['n_dense_layers']) +\
                 "-NN_" + str(n_neurons) + "-LR_" + str(learning_rate) + "-epochs_" + str(n_epochs) +"-" + UUID + ".keras"
    autoencoder_name = "ae_"+ modelTypeName + "-T_" + str(T) + "-LY_" + str(config['n_layers']) + "-DLY_" + str(config['n_dense_layers']) +\
                 "-NN_" + str(n_neurons) + "-LR_" + str(learning_rate) + "-epochs_" + str(n_epochs) +"-" + UUID + ".keras"

    experiment_id = mlflow.create_experiment(mlflow_exp_name)
    with mlflow.start_run(run_name=mlflow_exp_name, experiment_id=experiment_id) as mlflowrun:
        run_id = mlflowrun.info.run_id
    
        #autoencode
        autoencoder, training_results = create_autoencoder(config, X_train, Y_train, X_test, Y_test, autoencoder_name)
        try:
            y_predict_ae = autoencoder.predict(X_test, verbose=0)
            signature_ae = infer_signature(X_test, y_predict_ae)

            mlflow.tensorflow.log_model(autoencoder, autoencoder_name, 
                                            signature = signature_ae,
                                            #input_example=X_train[0].reshape(1, -1, 1), 
                                            registered_model_name = autoencoder_name)
            #save_model(encoder, autoencoder_name)

        except:
            print(f'Ray-MLFlow: Could not save model {autoencoder_name}')
        
        #forecaster
        X_train_ = np.concatenate([X_train, autoencoder.predict(X_train, verbose=0)], axis = -1)
        X_test_ = np.concatenate([X_test, autoencoder.predict(X_test, verbose=0)], axis = -1)
        
        x_train_shape = X_train_.shape[1:]
        nn_model = create_forecaster(config, x_train_shape)
        es_m = EarlyStopping(monitor='val_loss', mode='min', verbose=0, patience=5, restore_best_weights=True)
        mc_m = ModelCheckpoint(model_name, monitor='val_loss', mode='min', save_best_only=True)    
        train_results = nn_model.fit(X_train_, Y_train, epochs=n_epochs, validation_data=(X_test_, Y_test), 
                            callbacks=[es_m, mc_m], verbose=0)
            
        #evaluate and print results of forecaster
        try:
            saved_model = load_model(model_name)
        except:
            saved_model = nn_model
            save_model(nn_model, model_name)

        # Reconstruct signal using the train data and get the AE and SE errors to obtain the 3-sigma thresholds
        y_pred_train = saved_model.predict(X_train_, verbose=0)

        errors_ae_train = calculate_absolute_prediction_errors(Y_train, y_pred_train)
        lo_3sigma_ae, up_3sigma_ae = calculate_3sigma_threshold(errors_ae_train)
        anomalies_ae_train = calculate_3sigma_anomalies(errors_ae_train, lo_3sigma_ae, up_3sigma_ae)
        
        errors_se_train = calculate_squared_prediction_errors(Y_train, y_pred_train)
        lo_3sigma_se, up_3sigma_se = calculate_3sigma_threshold(errors_se_train)
        anomalies_se_train = calculate_3sigma_anomalies(errors_se_train, lo_3sigma_se, up_3sigma_se)
    
        three_sigma_thresholds = {}
        three_sigma_thresholds['lo_3sigma_ae'] = lo_3sigma_ae
        three_sigma_thresholds['up_3sigma_ae'] = up_3sigma_ae
        three_sigma_thresholds['lo_3sigma_se'] = lo_3sigma_se
        three_sigma_thresholds['up_3sigma_se'] = up_3sigma_se
        
        # Test spike detection using 3-sigma rule on the test data        
        y_predict = saved_model.predict(X_test_, verbose=0)
        
        errors_ae = calculate_absolute_prediction_errors(Y_test, y_predict)
        anomalies_ae = calculate_3sigma_anomalies(errors_ae, lo_3sigma_ae, up_3sigma_ae)
        errors_se = calculate_squared_prediction_errors(Y_test, y_predict)
        anomalies_se = calculate_3sigma_anomalies(errors_se, lo_3sigma_se, up_3sigma_se)

        try:
            r2 = r2_score(Y_test, y_predict)
        except:
            r2 = 110
        if np.isnan(r2):
            r2 = 110

        try:
            mae = mean_absolute_error(Y_test, y_predict)
        except:
            mae = 100
        if np.isnan(mae):
            mae = 100

        try:
            mape = mean_absolute_percentage_error(Y_test, y_predict)
        except:
            mape = 100
        if np.isnan(mape):
            mape = 100

        try:
            mse = mean_squared_error(Y_test, y_predict)
        except:
            mse = 100
        if np.isnan(mse):
            mse = 100

        try:
            pcc = np.corrcoef(Y_test, y_predict.flatten())[0,1]
        except:
            pcc = 100
        if np.isnan(pcc):
            pcc = 100


        fig = plt.figure(figsize=(20,15))
        plt.title("Spikes Train Data")
        plt.plot(Y_train,label="Original Data", alpha=0.6, c='gray')
        plt.plot(y_pred_train,label="Predict 1-step Forecast", alpha=0.6, c='red', linewidth=3)
        plt.scatter(np.where(anomalies_ae_train==True), y_pred_train[np.where(anomalies_ae_train==True)], 
                    alpha=0.8, color='green', s=350, label="3-Sigma Anomalies AE")
        plt.scatter(np.where(anomalies_se_train==True), y_pred_train[np.where(anomalies_se_train==True)], 
                    alpha=0.8, color='magenta', s=150, label="3-Sigma Anomalies SE")
        plt.legend()
        figName = f"Y_train_spikes-T_{T}.png"
        mlflow.log_figure(fig, figName)
        #plt.savefig(figName, transparent=False)
        fig.clf()
        plt.close()
    
        fig = plt.figure(figsize=(20,15))
        plt.title("Spikes Test Data T=" + str(T) + " with predict 1 on "+ str(model_exp) +": NN=" + str(n_neurons) + " epochs=" + str(n_epochs) +
                  " lr=" + str(learning_rate))
        plt.plot(y_predict,label="Predict 1-step Forecast", alpha=0.6, c='red', linewidth=3)
        plt.plot(Y_test,label="Original Data", alpha=0.6, c='black')
        plt.scatter(np.where(anomalies_ae==True), y_predict[np.where(anomalies_ae==True)], 
                    alpha=0.8, color='green', s=350, label="3-Sigma Anomalies AE")
        plt.scatter(np.where(anomalies_se==True), y_predict[np.where(anomalies_se==True)], 
                    alpha=0.8, color='magenta', s=150, label = "3-Sigma Anomalies SE")
        plt.legend()    
        figName = f"Y_predict-1-step-spikes-T_{T}.png"
        mlflow.log_figure(fig, figName)
        #plt.savefig(figName, transparent=False)
        fig.clf()
        plt.close()
            
        try:
            signature = infer_signature(X_test_, y_predict)
    
            mlflow.tensorflow.log_model(nn_model, model_name, 
                                            signature = signature,
                                            registered_model_name = model_name)
        except:
            print(f'Ray-MLFlow: Could not save model {model_name}')
            
        try:
            mlflow.log_dict(three_sigma_thresholds, "three_sigma_thresholds.json")
            mlflow.log_param("n_layer_size", n_neurons)
            mlflow.log_param("n_layers", config['n_layers'])
            mlflow.log_param("n_dense_layers", config['n_dense_layers'])
            mlflow.log_param("activation_fn", config['activation'])
            mlflow.log_param("epochs", n_epochs)
            mlflow.log_param("learning_rate", learning_rate)
            mlflow.log_param("optimizer", "adam")
            mlflow.log_param("time_window", config['T'])
            mlflow.log_param("dense_dp", config['dense_dp'])
            mlflow.log_param("model_exp", model_exp)
            
            mlflow.log_metric("mae", mae)
            mlflow.log_metric("mse", mse)
            mlflow.log_metric("mape", mape)
            mlflow.log_metric("r2_score", r2)
            mlflow.log_metric("pearson_corr_coef", pcc)
            
        except:
            exception_param_metric_dict = {}
            log_metric_dict = {
                'r2_score': r2,
                'mae': mae,
                'mape': mape,
                'mse': mse,
                'pcc': pcc
            }        
            log_param_dict = {
                "n_layer_size": n_neurons,
                "n_layers": config['n_layers'],
                "n_dense_layers": config['n_dense_layers'],
                "activation_fn": config['activation'],
                "epochs": n_epochs,
                "learning_rate": learning_rate,
                "optimizer": "adam",
                "time_window": config['T'],
                "dense_dp": config['dense_dp'],
                "model_exp": model_exp            
            }
            exception_param_metric_dict['log_param_dict'] = log_param_dict
            exception_param_metric_dict['log_metric_dict'] = log_metric_dict
            mlflow.log_dict(exception_param_metric_dict, "exception_param_metric_dict.json")

    train.report({"mse":mse, "mae":mae, "mape":mape, "r2":r2, "autoencoder_name":autoencoder_name, "model_name":model_name, "T":T, "run_id":run_id}) # for Ray>=2.7
    
    #air.session.report({"mse":mse, "mae":mae, "mape":mape, "r2":r2, "autoencoder_name":autoencoder_name, "model_name":model_name, "T":T, "run_id":run_id})

In [ ]:
def tune_model(num_training_iterations, num_samples):
    sched = AsyncHyperBandScheduler(
        time_attr="training_iteration", max_t=10, grace_period=5
    )
    
    #we have a cluster of 10 worker nodes with requests 1 CPU and limits 2 CPU settings for the pod
    resource_group = tune.PlacementGroupFactory([{'CPU': 1.0}] * 2) 
    tuner = tune.Tuner(
        tune.with_resources(train_model, resources=resource_group),
        tune_config=tune.TuneConfig(
            metric="mse",
            mode="min",
            scheduler=sched,
            num_samples=num_samples,
            max_concurrent_trials=10,
        ),
        run_config=air.RunConfig(
            name=modelTypeName,
            verbose = 1,
            stop={"training_iteration": num_training_iterations},
        ),
        param_space=param_space
    )
    
    results = tuner.fit()
    return results    

In [ ]:
@ray.remote
def run_n_step_evaluation(model_name, autoencoder_name, run_id, T, predict_ahead):
    import mlflow

    mlflow.set_tracking_uri(MLFLOW_URI)
    mlflow.set_registry_uri(MLFLOW_URI)

    model = mlflow.tensorflow.load_model(f'models:/{model_name}/1')
    autoencoder = mlflow.tensorflow.load_model(f'models:/{autoencoder_name}/1')

    params = mlflow.get_run(run_id).to_dictionary()['data']['params']
    n_neurons = params['n_layer_size']
    learning_rate = params['learning_rate']
    n_epochs = params['epochs']
    model_exp = params['model_exp']
    
    n_step_metrics = {}
    
    X_train, Y_train, X_test, Y_test = prepare_dataset(dataFrame[dataColumnName], T)
    
    #autoencode input
    X_test_ = np.concatenate([X_test, autoencoder.predict(X_test, verbose=0)], axis = -1)
    
    y_predict = model.predict(X_test_, verbose=0)
    y_pred_nsteps, avg_ae, avg_se = generate_nsteps_forecast(X_test_, Y_test, model, autoencoder, predict_ahead)
    
    errors_ae2 = calculate_absolute_prediction_errors(Y_test, y_pred_nsteps)
    errors_se2 = calculate_squared_prediction_errors(Y_test, y_pred_nsteps)

    try:
        r2_nStep = r2_score(Y_test, y_pred_nsteps)
    except:
        r2_nStep = 100

    try:
        mae_nStep = mean_absolute_error(Y_test, y_pred_nsteps)
    except:
        mae_nStep = 100

    try:
        mape_nStep = mean_absolute_percentage_error(Y_test, y_pred_nsteps)
    except:
        mape_nStep = 100

    try:
        mse_nStep = mean_squared_error(Y_test, y_pred_nsteps)
    except:
        mse_nStep = 100

    try:
        pcc_nStep = np.corrcoef(Y_test, y_pred_nsteps.flatten())[0,1]
    except:
        pcc_nStep = 100

    crt_step = f'predict_ahead_{predict_ahead}'
    n_step_metrics[crt_step] = {
                                    'r2_nStep': r2_nStep,
                                    'mae_nStep': mae_nStep,
                                    'mape_nStep': mape_nStep,
                                    'mse_nStep': mse_nStep,
                                    'pcc_nStep': pcc_nStep,
                                    'agv_ae_at_nStep': avg_ae,
                                    'agv_se_at_nStep': avg_se        
                                   }

    with mlflow.start_run(run_id=run_id, nested=True) as run:
        artifact_uri = run.info.artifact_uri
        three_sigma_thresholds = mlflow.artifacts.load_dict(artifact_uri + "/three_sigma_thresholds.json")        

        anomalies_ae2 = calculate_3sigma_anomalies(errors_ae2, 
                                                   three_sigma_thresholds['lo_3sigma_ae'], 
                                                   three_sigma_thresholds['up_3sigma_ae'])
        anomalies_se2 = calculate_3sigma_anomalies(errors_se2, 
                                                   three_sigma_thresholds['lo_3sigma_se'], 
                                                   three_sigma_thresholds['up_3sigma_se'])
        
        fig = plt.figure(figsize=(20,15))
        plt.title("Compare forecasts T=" + str(T) + " predict_ahead=" + str(predict_ahead) + " with predict 1" + 
                 " for AE-LSTM "+ modelTypeName +": NN=" + str(n_neurons) + " LR= " + str(learning_rate) + " epochs=" + str(n_epochs))
        plt.plot(Y_test,label="Original Data", alpha=0.6, c='red',linewidth=2)
        plt.plot(y_predict,label="Predicted Data 1-step", alpha=0.6, c='black', linewidth=2)
        plt.plot(y_pred_nsteps,label="Predicted Data " + str(predict_ahead) + "-steps", alpha=0.6, c='blue', linewidth=2)
        plt.legend()
        figName = f"compare-forecasts-1_{predict_ahead}.png"
        mlflow.log_figure(fig, figName)
        #plt.savefig(figName, transparent=False)
        fig.clf()
        plt.close()
        
        fig = plt.figure(figsize=(20,15))        
        plt.title("Predict Spikes T=" + str(T) + " with predict " + str(predict_ahead) + " on " + str(model_exp) + ": NN=" 
                  + str(n_neurons) + " epochs=" + str(n_epochs) + " lr=" + str(learning_rate))
        plt.plot(y_pred_nsteps,label="Predict " + str(predict_ahead) + "-step Forecast", alpha=0.6, c='red', linewidth=3)
        plt.plot(Y_test,label="Original Data", alpha=0.6, c='black')
        plt.scatter(np.where(anomalies_ae2==True), y_pred_nsteps[np.where(anomalies_ae2==True)], 
                    alpha=0.8, color='green', s=350, label="Anomalies AE")
        plt.scatter(np.where(anomalies_se2==True), y_pred_nsteps[np.where(anomalies_se2==True)], 
                    alpha=0.8, color='magenta', s=150, label = "Anomalies SE")
        plt.legend();    
        figName = f"Y-predict-spikes-step-{predict_ahead}-with-T_{T}.png"
        mlflow.log_figure(fig, figName)
        #plt.savefig(figName, transparent=False)
        fig.clf()
        plt.close()

        fname = f'{predict_ahead}-step-metric.json'
        mlflow.log_dict(n_step_metrics, fname)
        
    return n_step_metrics
    

In [ ]:
####
# Set the target environment to cluster or local. Default is local
####
run_local = True

In [ ]:
if run_local == False:
    print("Running on Cluster")
    # Run this cell only if you are using RHOAI as training environment. Otherwise run only the next one
    from codeflare_sdk import TokenAuthentication
    from codeflare_sdk import generate_cert
    
    auth = TokenAuthentication(
        token = "<your_access_token>", # execute ocp whoami -t on the authenticated cluster to obtain the token
        server = "<OCP cluster API URL>",
        skip_tls = True
    )
    auth.login()
    
    # Create required TLS cert and export the environment variables to enable TLS
    generate_cert.generate_tls_cert('raycluster-complete', 'raycluster')
    generate_cert.export_env('raycluster-complete', 'raycluster')
    
    ray_endpoint = 'ray://raycluster-complete-head-svc.raycluster.svc.cluster.local:10001' # ensure your ray cluster URL is correct
    ray.shutdown()
    ray.init(address=ray_endpoint, logging_level=logging.ERROR, log_to_driver=False)
else:
    ray.shutdown()
    ray.init()    

In [ ]:
results_lstm_ae = tune_model(1, 10)

In [ ]:
result = results_lstm_ae.get_best_result()
result

In [ ]:
results_n_step_predict = {}
#for result in results_lstm_ae:
try:
    model_name = result.metrics['model_name']
    autoencoder_name = result.metrics['autoencoder_name']
    T = result.metrics['T']
    run_id = result.metrics['run_id']
    for predict_ahead in [5, 10, 15, 30, 45, 60, 90, 120]: 
    #for predict_ahead in [5, 10]: 
        res = run_n_step_evaluation.remote(model_name, autoencoder_name, run_id, T, predict_ahead)
        #res = run_n_step_evaluation(model_name, autoencoder_name, run_id, T, predict_ahead)
        tag = f'runID_{run_id}-model_{model_name}-T_{T}-p_ahead_{predict_ahead}'
        results_n_step_predict[tag] = res
        print(f'Scheduled job for T:{T}, predict_ahead: {predict_ahead}, model_name:{model_name}, run_id:{run_id}')    
except:
    print(f'ERROR scheduling job for T:{T}, predict_ahead: {predict_ahead}, model_name:{model_name}, run_id:{run_id}')

In [ ]:
ex_n_step = {}
for item in results_n_step_predict.keys():
    try:
        res = ray.get(results_n_step_predict[item])
    except:
        print(f'Error getting results for key:{item}')
        res = None
        
    ex_n_step[item]=res

In [ ]:
len(ex_n_step.keys())

In [ ]:
ex_n_step

In [ ]:
ray.shutdown()

In [ ]:
end_time = datetime.datetime.now()

In [ ]:
time_difference = (end_time - start_time).total_seconds()

In [ ]:
print(f"Experiment runtime was {time_difference} seconds")